In [1]:
from scipy.special import binom as binomial
#compute probabilities of for given branch in the binary tree
def split_weights(n,k):
    v0=binomial(n-1,k)
    v1=binomial(n-1,k-1)
    return v0/(v0+v1),v1/(v0+v1)

#compute all probabilities for the binary tree
def circuit_layout(n,k):
    L=[]
    for i in range(n):
        for j in range(min(i,k-1),-1,-1):
            if n-i >= k-j:
                L.append([n-i,k-j,split_weights(n-i,k-j)])
    return L

# Define all the Helper Functions
(Everything as before)

In [2]:
def set_ancillas_to_num(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    

In [3]:
def add_negates_for_check(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==0:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    for i in range(len(bin(num)[2:]),len(ancillas)):
        c.add(gates.X(ancillas[i]))
    

In [4]:
def controlls_for_check_num(ancillas,num):
    l=[]
    c=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            l.append(ancillas[c])
        c+=1
    return l

In [5]:
def sub_one(c,ancillas,controlls):
    a=ancillas
    c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in range(1,len(a)):
        controlls.append(a[i-1])
        c.add(gates.X(a[i]).controlled_by(*controlls))
        

In [6]:
def add_one(c,ancillas,controlls):
    a=ancillas
    for i in range(len(a)-1):
        controlls.append(a[i])

    #c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in reversed(range(0,len(a))):
        c.add(gates.X(a[i]).controlled_by(*controlls))
        controlls.pop()

In [7]:
def row_add(c,col,row_res,row_addend,H,n,controlls):
    for i in range(col+1,n+1):
        g=gates.X(H[row_res,i])
        controlls.append(H[row_addend,i])
        c.add(g.controlled_by(*controlls))
        controlls.pop()
        
    

In [8]:
def add_pivot(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    c.add(gates.X(H[i,p]).controlled_by(*controlls))
    #loop for differnt rows in which the pivot is searched
    for l in range(r-(i+1)):
        #current row in which pivot is searched
        pr=i+l+1
        #need to add controls on permutation
        row_add(c,p,i,pr,H,n,[H[i,p]]+[H[i+1+o,p] for o in range(l+1)]+controlls)
        #add not for "if all preious were 0"-statement
        if l!=r-i-2:
            c.add(gates.X(H[pr,p]).controlled_by(*controlls))
    #revert all nots
    for l in range(r-i-1):
        c.add(gates.X(H[i+l,p]).controlled_by(*controlls))
    

In [9]:
def lower_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each element below the current diagonal
    for l in range(r-i-1):
        #add row if needed to eliminate ones in column p
        row_add(c,p,i+l+1,i,H,n,[H[i+l+1,p]]+controlls)

In [10]:
def upper_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each row above the current diagonal entry (identiy)
    for l in range(i):
        #eliminate ones by adding row[i] to that row
        c.add(gates.X(H[l,n]).controlled_by(*([H[l,p],H[i,n]]+controlls)))

# Create Qibo circuit with less qubits
(only log(r) ancillas and (n-r)*r for the matrix

In [43]:
from math import log2,ceil
from qibo.models import Circuit
from qibo import gates
import numpy as np

r=4
n=7

In [44]:

def create_qibo_circuit(n,r):
    n_anc=int(ceil(log2(r+1)))
    c=Circuit(int(n+n_anc+(n-r+1)*r))
    L=circuit_layout(n,r)

    anc=[i for i in range(n,n+n_anc)]
    set_ancillas_to_num(c,anc,r)
    tmp=n

    for i in L:
        if tmp!=i[0]:
            sub_one(c,anc,[n-tmp])
            tmp=i[0]
        
        if(i[2]==(0,1)):
            add_negates_for_check(c,anc,i[1])
            c.add(gates.X(n-i[0]).controlled_by(*anc))
            add_negates_for_check(c,anc,i[1])    
        else:
            if i[0]!=n:
                add_negates_for_check(c,anc,i[1])
                c.add(gates.RY(n-i[0],float(2*np.arccos(np.sqrt(i[2][0])))).controlled_by(*anc))
                add_negates_for_check(c,anc,i[1])
            else:
                c.add(gates.RY(0,float(2*np.arccos(np.sqrt(i[2][0])))))
    sub_one(c,anc,[n-1])
    return c

In [45]:
c=create_qibo_circuit(n,r)
c.add(gates.M(*[i for i in range(n)]))
measure=c(nshots=1000)

print(c.draw())
for i in measure.frequencies():
    print(i,measure.frequencies()[i])


q0 : ─RY─o─o─o────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M─
q1 : ────|─|─|───RY───RY───o─o─o──────────────────────────────────────────────────────────────────────────────────────────────────────────────────M─
q2 : ────|─|─|───|────|────|─|─|───RY─────RY───RY───o─o─o─────────────────────────────────────────────────────────────────────────────────────────M─
q3 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───RY─────RY─────RY───X───o─o─o──────────────────────────────────────────────────────────M─
q4 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──────|────|───|─|─|───RY─────RY─────X───o─o─o────────────────────────────────M─
q5 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──────|────|───|─|─|───|──────|──────|───|─|─|───RY─────X───o─o─o─────────────M─
q6 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──────|────|───|─|─|───|──────|──────|

### Providing custom Gaussian elimination for GF2
(needed for preparing the matrix, but I would also suggest using this function instead of np.linalg.solve() as the later one caused some problems for certain parameters)

In [46]:
def solve_gf2(A,t):
    r=A.shape[0]
    b=np.copy(t)
    for i in range(r):    
        for j in range(i+1,r):
            if A[j,i]==1:
                if A[i,i]!=1:
                    A[i]^=A[j]
                    b[i]^=b[j]
                A[j]^=A[i]
                b[j]^=b[i]
        if A[i,i]!=1:
            raise ValueError("not invertible")
    for i in reversed(range(r)):
        for j in range(i):
            if A[j,i]==1:
                A[j]^=A[i]
                b[j]^=b[i]
    return b



# Instance Generation
(mostly as before, but only accepting non-zero vectors b and matrices where the first k columns form and invertible matrix and using custom gaussian instead of np.linalg.solve)

In [47]:
import itertools

solutions=[]

while(len(solutions)<4):
    solutions=[]
    A=np.random.randint(2, size=(r, n))
    b=np.random.randint(2, size=(r,1))
    if all(b[i]==0 for i in range(r)):
        continue
    comb=itertools.combinations([i for i in range(n)],r)
    Aprime=np.arange(r*r).reshape(r,r)
    first=1
    for i in comb:
        ind=0
        for j in i:
            for l in range(r):
                Aprime[l,ind]=A[l,j]
            ind+=1
        mask=0
        for j in i:
            mask^=(1<<(n-j-1))
        v=bin(mask)[2:]
        while (len(v)<n):
            v="0"+v
        try:
            sol=solve_gf2(Aprime,b)#np.linalg.solve(Aprime,b)%2
            for it in reversed(sol):
                if it==0:
                    v="0"+v
                else:
                    v="1"+v
            solutions.append(v)
        except:
            pass
        if len(solutions)==0 and first==1:
            break
        frist=0

print("solve all possible linear systems, where" )
print("")
print("H =")
print(A)
print("")
print("b =")
print(b)
print("")
print("solutions =")
print(solutions)


solve all possible linear systems, where

H =
[[1 0 1 0 0 0 0]
 [0 1 0 1 0 0 1]
 [1 0 1 1 0 0 1]
 [0 1 1 0 1 1 1]]

b =
[[1]
 [1]
 [1]
 [1]]

solutions =
['11001111000', '11001110001', '11001101100', '11001101010', '11001101001', '11001100101', '11001100011', '11010111100', '11010111010', '11110111001', '11100110101', '11100110011']


### Prepare the matrix 
(precompute Gaussian on the first r columns)

In [48]:
b=solve_gf2(A,b)
A,b

(array([[1, 0, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 0, 1]]),
 array([[1],
        [1],
        [0],
        [0]]))

### Swap row function 
(needed to handle the first r columns)

In [49]:
def swap_rows(c,i,j,H,n,r,controlls):
    for col in range(n-r+1):
        c.add(gates.SWAP(H[i,col],H[j,col]).controlled_by(*controlls))

### Adjust qubit access matrix to smaller setting
(ancilla register smaller and smaller input matrix)

In [50]:

perm=[i for i in range(n)]
n_anc=int(ceil(log2(r+1)))
ancillas=[i for i in range(n,n+n_anc)]

H=np.arange((n-r+1)*r).reshape(r,n-r+1)
c=n+n_anc

for j in range(n-r+1):
    for i in range(r):
        H[i,j]=c
        c+=1


### Create circuit

In [51]:
c=create_qibo_circuit(n,r)

#initialize matrix in quantum register

for i in range(r):
### Only n-r columns now ###
    for j in range(n-r):
        ### last n-r columns of A need to be selected, so we have an offset of r now ###
        if A[i, j+r] == 1:
            c.add(gates.X(H[i, j]))
        ################
for i in range(r):
    if b[i] == 1:
        c.add(gates.X(H[i, n-r]))
        
#add pivots and create upper triangular matrix
for j in range(r-1):
    set_ancillas_to_num(c,ancillas,r)
    for i in range(n):
        if j<i:
            add_negates_for_check(c,ancillas,r-j)
            ### new ###
            if i<r:
                swap_rows(c,i,j,H,n,r,ancillas+[perm[i]])
            #########
            ### here the input to add_pivot and lower_col_elimination is now i-r and n-r instead of i and n respectively ###
            else:
                add_pivot(c,j,i-r,H,n-r,r,ancillas+[perm[i]])
                lower_col_elimination(c,j,i-r,H,n-r,r,ancillas+[perm[i]])
            #############
            add_negates_for_check(c,ancillas,r-j)
        sub_one(c,ancillas,[perm[i]]) 

#perform backsubstitution
for j in range(r-1):
    set_ancillas_to_num(c,ancillas,r)
    for i in reversed(range(n)):
        ### i>=r restriction is new and again input to upper_col_elimination is now i-r and n-r instead of i and n ###
        if r-j-1<=i and i>=r:
            add_negates_for_check(c,ancillas,r-j)
            upper_col_elimination(c,r-j-1,i-r,H,n-r,r,ancillas+[perm[i]])
            add_negates_for_check(c,ancillas,r-j)
        #########################
        sub_one(c,ancillas,[perm[i]])

### syndrome is now in column n-r of H ###
for i in range(r):
    add_one(c,ancillas[::-1],[H[i][n-r]])
########

#add measurements
c.add(gates.M(*([H[i,n-r] for i in range(r)]+perm)))




In [52]:
print(c.draw(line_wrap=150))

q0 : ─RY─o─o─o──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────o─ ...
q1 : ────|─|─|───RY───RY───o─o─o────────────────────────────────────────────────────────────────────────────────────────────────────────────────────|─ ...
q2 : ────|─|─|───|────|────|─|─|───RY─────RY───RY───o─o─o───────────────────────────────────────────────────────────────────────────────────────────|─ ...
q3 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───RY─────RY─────RY───X───o─o─o────────────────────────────────────────────────────────────|─ ...
q4 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──────|────|───|─|─|───RY─────RY─────X───o─o─o──────────────────────────────────|─ ...
q5 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──────|────|───|─|─|───|──────|──────|───|─|─|───RY─────X───o─o─o───────────────|─ ...
q6 : ────|─|─|───|────|────|─|─|───|──────|────|────|─|─|───|──────|──

### Check if circuit solves all solveable linear systems correctly

In [ ]:

m=c(nshots=10000)
m.frequencies().keys()
if all(i in m.frequencies().keys() for i in solutions):
    print("Circuit works correct")
else:
    print("Circuit works erroneous")